In [6]:
import chromedriver_autoinstaller

In [7]:
# pip install selenium
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd
import re
from selenium.common.exceptions import *


In [8]:
def search_youtube_videos(keywords):
    url = f"https://www.youtube.com/results?search_query={(keywords)}"

    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(5)

    wait = WebDriverWait(driver, 10)
    cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
    cookie_consent.click()

    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(3)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

    rows = []
    for video in videos:
        name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
        link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
        info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
        try:
            views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)
        except:
            views = 'unknown'

        try:
            date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
        except:
            date = 'Unknown'

        try:
            duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec|hour)s?(?:,\s*)?)+\d*\s*(?:minute|second?)s?', info[1]).group()
        except:
            duration = 'Unknown'

        try:
            description = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
        except:
            description = 'Unknown'

        rows.append([name, views, duration, date, description, link])

    df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'description', 'link'])
    
    driver.quit()

    return df_yt


In [21]:
def scrape_comments(df):
    comments = []

    for i, row in df.iterrows():
        url = row['link']
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(30)
        try:
            wait = WebDriverWait(driver, 10)
            cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
            cookie_consent.click()
        except TimeoutException:
            print('Cookie pop-up not loaded')

        time.sleep(30)
        
        driver.execute_script("window.scrollBy(0, 500);")

        try:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.XPATH, '//ytd-comment-thread-renderer')))
            time.sleep(30)
            while True:
                scroll_height = 10
                document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
                driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
                time.sleep(2)
                document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
                if document_height_after == document_height_before:
                    break

            html = driver.page_source
            try:
                soup = BeautifulSoup(html, "html.parser")

                for comment in soup.find_all('ytd-comment-thread-renderer'):
                    text = comment.find('yt-formatted-string', {'class': 'style-scope ytd-comment-renderer'}).text
                    likes = comment.find('span', {'id': 'vote-count-left'}).text.strip() if comment.find('span', {'id': 'vote-count-left'}) else 0
                    replies = comment.find('button', {'class': 'yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text'})
                    if replies:
                        num_comments = replies.get('aria-label').split()[0]
                    else:
                        num_comments = 0
                    username_element = comment.find('a', {'id': 'author-text'}).find('span', {'class': 'style-scope ytd-comment-renderer'})
                    if username_element:
                        username = username_element.text.replace('[','').replace(']','').replace('\n','')
                    else:
                        username = ''
                    link = url
                    try:
                        date = comment.find('a', class_='yt-simple-endpoint style-scope yt-formatted-string')
                    except:
                        date = 'Unknown'
            except:
                print('Fail')

                comments.append([username, text, likes, num_comments, date, link])
        except TimeoutException:
            comments.append(["ERROR", 'ERROR', 'ERROR', 'ERROR', 'ERROR', url])
        finally:
            driver.quit()
            print(f'{url} done')

        df_comments = pd.DataFrame(comments, columns=['Username', 'Comment', 'Likes', 'Replies', 'Date', 'link'])

    return df_comments


In [4]:
df_yt = search_youtube_videos('artificial intelligence')

In [4]:
df_yt = pd.read_csv('df_yt.csv')
df_yt_test = df_yt.iloc[547:548]

In [12]:
df_yt_test

,Unnamed: 0,Name,views,duration,date,description,link
547,547,Paragraph Writing from hints || 10th writing s...,64,"12 minutes, 55 seconds",14 minutes ago,Paragraph Writing from hints || 10th writing s...,https://www.youtube.com/watch?v=e8MY1yvqySo


In [18]:
scrape_comments(df_yt_test)

https://www.youtube.com/watch?v=e8MY1yvqySo done


,Username,Comment,Likes,Replies,Date,link
0,Shahbaz Clicks,Tommorov is our first paper best of luck to ev...,11,0,1 day ago,https://www.youtube.com/watch?v=e8MY1yvqySo
1,FACTUAL MATHS,Can u tell me plzz which one App are you using...,1,0,2 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
2,POKEMON HERO,Thanks a lot ️,1,0,3 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
3,Rakeeb,And in play I have learnt the them of proposa...,1,0,2 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
4,Cricket With Sher,,3,0,3 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
5,Mr Dott,Sir Paragraph writing mai 1 hi para mai likhna...,1,0,3 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
6,Mir Kamran,Thank you sir,1,0,3 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
7,m amin malik,Thnks sir,1,0,2 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
8,Khalid Subzar,Thanks a lot sir,1,1,3 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo
9,Azan Azan,️️,1,0,3 days ago,https://www.youtube.com/watch?v=e8MY1yvqySo


In [4]:
df_yt = pd.read_csv('df_yt.csv')

In [3]:

df_yt = pd.read_csv('df_yt.csv')

In [6]:
df_yt.to_csv('df_yt.csv', index = True)


In [6]:
df_scraped_comments = scrape_comments(df_yt)

NameError: name 'df_yt' is not defined

In [10]:
df_yt = pd.read_csv('df_yt.csv')

In [11]:
df_yt2  = df_yt.iloc[0:10].reset_index()
df_yt3  = df_yt.iloc[25:45].reset_index()
df_yt3  = df_yt.iloc[45:70].reset_index()
df_yt4  = df_yt.iloc[70:100].reset_index()
df_yt5  = df_yt.iloc[100:200].reset_index()
df_yt6  = df_yt.iloc[200:300].reset_index()
df_yt7  = df_yt.iloc[0:10].reset_index()
df_yt8  = df_yt.iloc[25:45].reset_index()
df_yt9  = df_yt.iloc[300:400].reset_index()
df_yt10  = df_yt.iloc[400:500].reset_index()

In [7]:
for i in range(2, 11):
    globals()[f"df_yt{i}"].drop(columns=['Unnamed: 0', 'index'], inplace=True)
df_yt2 

,Name,views,duration,date,description,link
0,What Is Artificial Intelligence? | 🔥Artificial...,"1,166,565","5 minutes, 28 seconds",3 years ago,"This video on ""What is Artificial Intelligence...",https://www.youtube.com/watch?v=ad79nYk2keg
1,How will AI change the world?,"730,358","5 minutes, 56 seconds",3 months ago,Explore the current limitations of artificial ...,https://www.youtube.com/watch?v=RzkD_rTEBYs
2,Artificial Intelligence: Last Week Tonight wit...,"5,225,357",Unknown,11 days ago,Artificial intelligence is increasingly becomi...,https://www.youtube.com/watch?v=Sqa8Zo2XWc4
3,"Midweek Marvels: Pete Buttigieg, Marianne Will...","16,533",Unknown,1 day ago,For entertainment purposes only. Just one guy'...,https://www.youtube.com/watch?v=H_xReIQ8DLo
4,The Danger of AI | Scary Technology | Artifici...,"75,913",Unknown,2 weeks ago,The Danger of AI - An examination of an altern...,https://www.youtube.com/watch?v=5guiUrgcblM
5,Is artificial intelligence coming for your job?,"2,711","3 minutes, 29 seconds",1 day ago,For decades people have worried about the rise...,https://www.youtube.com/watch?v=DPyzVtMs-Vo
6,Artificial intelligence and algorithms: pros a...,"10,011,560",Unknown,3 years ago,Developments in artificial intelligence (AI) a...,https://www.youtube.com/watch?v=s0dMTAQM4cw
7,Steven Spielberg Talks Artificial Intelligence...,"63,660","3 minutes, 37 seconds",2 days ago,In this bonus scene from Stephen Colbert's int...,https://www.youtube.com/watch?v=tSoCtw5vNS0
8,How Google Is Getting Into the Artificial Inte...,"5,870","9 minutes, 29 seconds",1 day ago,Google Vice President for Trust and Safety Lau...,https://www.youtube.com/watch?v=uFyxM0Jlc-c
9,"ChatGPT: Artificial Intelligence, chatbots and...","194,439","13 minutes, 22 seconds",4 days ago,"Lesley Stahl speaks with Brad Smith, president...",https://www.youtube.com/watch?v=1wzPr4cUoMQ


In [11]:
def search_youtube_videos(keywords):
    url = f"https://www.youtube.com/results?search_query={(keywords)}"

    options = webdriver.ChromeOptions()
    options.add_argument("--incognito")
    #options.add_argument("--headless")
    options.add_argument("--disable-gpu")

    driver = webdriver.Chrome(options=options)

    driver.get(url)
    time.sleep(5)

    wait = WebDriverWait(driver, 10)
    cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
    cookie_consent.click()

    while True:
        scroll_height = 10
        document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
        driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
        time.sleep(3)
        document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
        if document_height_after == document_height_before:
            break

    html = driver.execute_script("return document.body.outerHTML;")
    soup = BeautifulSoup(html, "html.parser")

    videos = soup.find_all("div", class_="text-wrapper style-scope ytd-video-renderer")

    rows = []
    for video in videos:
        name = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").text
        link = "https://www.youtube.com" + video.find("a", class_="yt-simple-endpoint style-scope ytd-video-renderer")['href']
        info = video.find("yt-formatted-string", class_="style-scope ytd-video-renderer").get("aria-label").split(" by ")
        try:
            views = re.search(r'([\d,]+)\s*(views|view)', info[1]).group(1)
        except:
            views = 'unknown'

        try:
            date = re.search(r'(?P<count>\d+)\s+(?P<unit>\w+)\s+ago', info[1]).group()
        except:
            date = 'Unknown'

        try:
            duration =  re.search(r'\d+\s+(?:(?:minute|min|second|sec|hour)s?(?:,\s*)?)+\d*\s*(?:minute|second?)s?', info[1]).group()
        except:
            duration = 'Unknown'

        try:
            description = video.find("yt-formatted-string", class_="metadata-snippet-text style-scope ytd-video-renderer").text
        except:
            description = 'Unknown'

        rows.append([name, views, duration, date, description, link])

    df_yt = pd.DataFrame(rows, columns=['Name', 'views', 'duration', 'date', 'description', 'link'])
    
    driver.quit()

    return df_yt


In [53]:
df_scraped_comments = pd.concat([df_scraped_comments, df_scraped_comments2, df_scraped_comments3, df_scraped_comments4, df_scraped_comments5])

# reset index
df_scraped_comments = df_scraped_comments.reset_index(drop=True)

# export as CSV
df_scraped_comments.to_csv('df_scraped_comments.csv', index=False)

In [10]:
scraped1 = scrape_comments(df_yt2)

https://www.youtube.com/watch?v=ad79nYk2keg done
https://www.youtube.com/watch?v=RzkD_rTEBYs done
https://www.youtube.com/watch?v=Sqa8Zo2XWc4 done
https://www.youtube.com/watch?v=H_xReIQ8DLo done
https://www.youtube.com/watch?v=5guiUrgcblM done
https://www.youtube.com/watch?v=DPyzVtMs-Vo done
https://www.youtube.com/watch?v=s0dMTAQM4cw done
https://www.youtube.com/watch?v=tSoCtw5vNS0 done
https://www.youtube.com/watch?v=uFyxM0Jlc-c done
https://www.youtube.com/watch?v=1wzPr4cUoMQ done


In [11]:
scraped1.to_csv('df_yt_scraped1.csv', index = True)


In [12]:
scraped2 = scrape_comments(df_yt3)
scraped2.to_csv('df_yt_scraped2.csv', index = True)


https://www.youtube.com/watch?v=L0NBQzy39EQ done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=w-eQVDxdzqg done
https://www.youtube.com/watch?v=TIRdJyf6SB0 done
Cookie pop-up not loaded
https://www.youtube.com/shorts/qdHqwEi6xu0 done
https://www.youtube.com/watch?v=-PztBI2lG54 done
https://www.youtube.com/watch?v=JT5q7u7xxNU done
https://www.youtube.com/watch?v=TogDlICXfAU done
https://www.youtube.com/watch?v=vsty2r189QI done
https://www.youtube.com/watch?v=rJ1Qao09CFI done
https://www.youtube.com/watch?v=P5HNeahRYDM done
https://www.youtube.com/watch?v=dUhvvoVtpVE done
https://www.youtube.com/watch?v=USH9xtxz-vU done
https://www.youtube.com/watch?v=NRepEvmCCIg done
https://www.youtube.com/watch?v=sAUpoJixwZg done
https://www.youtube.com/watch?v=fvtrRGmv7aU done
https://www.youtube.com/watch?v=mUkTz_1uzSs done
https://www.youtube.com/watch?v=k0lOL5Q58Ls done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=JMLsHI8aV0g done
https://www.youtube.com/watch?v=iyb38dzetRk 

In [7]:
scraped3 = scrape_comments(df_yt4)
scraped3.to_csv('df_yt_scraped3.csv', index = True)


https://www.youtube.com/watch?v=-ZeeuDrknYc done
https://www.youtube.com/watch?v=xRZIdepFoTM done
https://www.youtube.com/watch?v=jqVlmaZIX4U done
https://www.youtube.com/watch?v=PATxJ-wwcUY done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=kh5dN72GTQ8 done
https://www.youtube.com/watch?v=mJeNghZXtMo done
https://www.youtube.com/watch?v=5J5bDQHQR1g done
https://www.youtube.com/watch?v=0yCJMt9Mx9c done
https://www.youtube.com/watch?v=pOmpqdlVCoo done
Cookie pop-up not loaded
https://www.youtube.com/shorts/tDf7PWdSA7I done
https://www.youtube.com/watch?v=qhYWn9-IFYE done
https://www.youtube.com/watch?v=xDdtOpRyXhI done
https://www.youtube.com/watch?v=sSPKt5xNHSU done
https://www.youtube.com/watch?v=-86zfUqS-L8 done
https://www.youtube.com/watch?v=jPhJbKBuNnA done
https://www.youtube.com/watch?v=R4iJMIgj6ZI done
https://www.youtube.com/watch?v=lmkIpBdSe5E done
https://www.youtube.com/watch?v=5q87K1WaoFI done
https://www.youtube.com/watch?v=RdAQnkDzGvc done
Cookie pop-up not lo

In [9]:
scraped4 = scrape_comments(df_yt5)
scraped4.to_csv('df_yt_scraped4.csv', index = True)


https://www.youtube.com/watch?v=TRzBk_KuIaM done
https://www.youtube.com/watch?v=vyit-1zKsZ4 done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=OEg6XuXVsdM done
https://www.youtube.com/watch?v=x153zVRPd7s done
https://www.youtube.com/watch?v=R4iJMIgj6ZI done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=03tPZ1mkKQU done
https://www.youtube.com/watch?v=oz2n-rQCWzI done
https://www.youtube.com/watch?v=gIVSJDVQRf8 done
https://www.youtube.com/watch?v=zhpiVCQJTw8 done
https://www.youtube.com/watch?v=019HlWMQKCE done
https://www.youtube.com/watch?v=tvAh0JZF2YE done
Cookie pop-up not loaded
https://www.youtube.com/shorts/JpBBY1HbT54 done
Cookie pop-up not loaded
https://www.youtube.com/shorts/bG4WzJv2u-U done
Cookie pop-up not loaded
https://www.youtube.com/shorts/an2TNoBdzbw done
https://www.youtube.com/watch?v=63yr9dlI0cU done
https://www.youtube.com/watch?v=kPJNE1p7uT0 done
https://www.youtube.com/watch?v=8RBwR0mk6C4 done
https://www.youtube.com/watch?v=uMWaOv9-oHo d

TypeError: object of type 'NoneType' has no len()

In [ ]:
scraped5 = scrape_comments(df_yt6)
scraped5.to_csv('df_yt_scraped5.csv', index = True)


https://www.youtube.com/watch?v=THJysHMi81c done
https://www.youtube.com/watch?v=EYeF2e2IKEo done
https://www.youtube.com/watch?v=-86zfUqS-L8 done
https://www.youtube.com/watch?v=ZCsqp6ueDbw done
https://www.youtube.com/watch?v=wdH2GE9cryo done
https://www.youtube.com/watch?v=DNDqM1f5nQQ done
Cookie pop-up not loaded
https://www.youtube.com/shorts/IMu7jZlcefU done
https://www.youtube.com/watch?v=_avhEwu0AX4 done
https://www.youtube.com/watch?v=5_Xh_49BpY0 done
https://www.youtube.com/watch?v=Ra3fv8gl6NE done
Cookie pop-up not loaded
https://www.youtube.com/shorts/JTr6_J-hK98 done
https://www.youtube.com/watch?v=u_TE42-uWD0 done
https://www.youtube.com/watch?v=kmeaG_BQZ7M done
https://www.youtube.com/watch?v=jZgB3HZ6au4 done
https://www.youtube.com/watch?v=VfBaFX7gVLk done
Cookie pop-up not loaded
https://www.youtube.com/shorts/e5CoeyeC30k done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=GwftxcJ18Jw done
https://www.youtube.com/watch?v=pNyo9CMiq4g done
https://www.youtube.c

In [15]:
df_yt7

,index,Unnamed: 0,Name,views,duration,date,description,link
0,0,0,What Is Artificial Intelligence? | 🔥Artificial...,"1,166,565","5 minutes, 28 seconds",3 years ago,"This video on ""What is Artificial Intelligence...",https://www.youtube.com/watch?v=ad79nYk2keg
1,1,1,How will AI change the world?,"730,358","5 minutes, 56 seconds",3 months ago,Explore the current limitations of artificial ...,https://www.youtube.com/watch?v=RzkD_rTEBYs
2,2,2,Artificial Intelligence: Last Week Tonight wit...,"5,225,357",Unknown,11 days ago,Artificial intelligence is increasingly becomi...,https://www.youtube.com/watch?v=Sqa8Zo2XWc4
3,3,3,"Midweek Marvels: Pete Buttigieg, Marianne Will...","16,533",Unknown,1 day ago,For entertainment purposes only. Just one guy'...,https://www.youtube.com/watch?v=H_xReIQ8DLo
4,4,4,The Danger of AI | Scary Technology | Artifici...,"75,913",Unknown,2 weeks ago,The Danger of AI - An examination of an altern...,https://www.youtube.com/watch?v=5guiUrgcblM
5,5,5,Is artificial intelligence coming for your job?,"2,711","3 minutes, 29 seconds",1 day ago,For decades people have worried about the rise...,https://www.youtube.com/watch?v=DPyzVtMs-Vo
6,6,6,Artificial intelligence and algorithms: pros a...,"10,011,560",Unknown,3 years ago,Developments in artificial intelligence (AI) a...,https://www.youtube.com/watch?v=s0dMTAQM4cw
7,7,7,Steven Spielberg Talks Artificial Intelligence...,"63,660","3 minutes, 37 seconds",2 days ago,In this bonus scene from Stephen Colbert's int...,https://www.youtube.com/watch?v=tSoCtw5vNS0
8,8,8,How Google Is Getting Into the Artificial Inte...,"5,870","9 minutes, 29 seconds",1 day ago,Google Vice President for Trust and Safety Lau...,https://www.youtube.com/watch?v=uFyxM0Jlc-c
9,9,9,"ChatGPT: Artificial Intelligence, chatbots and...","194,439","13 minutes, 22 seconds",4 days ago,"Lesley Stahl speaks with Brad Smith, president...",https://www.youtube.com/watch?v=1wzPr4cUoMQ


In [16]:
scraped7 = scrape_comments(df_yt7)
scraped7.to_csv('df_yt_scraped7.csv', index = True)


https://www.youtube.com/watch?v=ad79nYk2keg done
https://www.youtube.com/watch?v=RzkD_rTEBYs done


In [ ]:
scraped8 = scrape_comments(df_yt8)
scraped8.to_csv('df_yt_scraped8.csv', index = True)


https://www.youtube.com/watch?v=_tA5cinv0U8 done
https://www.youtube.com/watch?v=uTl8ZYImjBQ done
https://www.youtube.com/watch?v=JMUxmLyrhSk done
https://www.youtube.com/watch?v=KzvgUxTUQZ8 done
https://www.youtube.com/watch?v=2HYemjTSxp8 done
https://www.youtube.com/watch?v=w-Q9PWz_8sM done
https://www.youtube.com/watch?v=ad79nYk2keg done
https://www.youtube.com/watch?v=RzkD_rTEBYs done
https://www.youtube.com/watch?v=Yt14Wq7PXBc done
https://www.youtube.com/watch?v=0-xADccFDCs done
https://www.youtube.com/watch?v=_19pRsZRiz4 done
https://www.youtube.com/watch?v=SQfE2pCq5rU done
https://www.youtube.com/watch?v=zjbJyHUkmys done
https://www.youtube.com/watch?v=EL3s9wRmwNs done
https://www.youtube.com/watch?v=h7jo7scB5c8 done
https://www.youtube.com/watch?v=5Mn-ToR9fGI done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=ttIOdAdQaUE done
https://www.youtube.com/watch?v=DKYNkMu5QLE done
https://www.youtube.com/watch?v=j_QqhxQzp_o done
Cookie pop-up not loaded
https://www.youtube

In [13]:
scraped9 = scrape_comments(df_yt9)
scraped9.to_csv('df_yt_scraped9.csv', index = True)


Cookie pop-up not loaded
https://www.youtube.com/shorts/tBqdYod5Efw done
https://www.youtube.com/watch?v=CmEEEQsRhbA done
Cookie pop-up not loaded
https://www.youtube.com/shorts/tEZZ2gxeM18 done
Cookie pop-up not loaded
https://www.youtube.com/shorts/Q4JSxhoXFPU done
https://www.youtube.com/watch?v=7eh4d6sabA0 done
Cookie pop-up not loaded
https://www.youtube.com/shorts/fNXtBUU3zyo done
https://www.youtube.com/watch?v=Ntu8nNBL28o done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=7ffDUDjwz5E done
https://www.youtube.com/watch?v=S2FSGyI18bo done
https://www.youtube.com/watch?v=7JmoBvz3pmI done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=2Rjt7YRzd7I done
https://www.youtube.com/watch?v=rdexw3Rc5cw done
Cookie pop-up not loaded
https://www.youtube.com/shorts/d_RZv5ZYfZw done
https://www.youtube.com/watch?v=6gd_MBRyBWE done
https://www.youtube.com/watch?v=96-u9s6D16k done
https://www.youtube.com/watch?v=60qzgcJObyY done
Fail
https://www.youtube.com/watch?v=OR_ztO4n1

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=111.0.5563.65)
Stacktrace:
Backtrace:
	(No symbol) [0x007DDCE3]
	(No symbol) [0x007739D1]
	(No symbol) [0x00684DA8]
	(No symbol) [0x006812F7]
	(No symbol) [0x00678835]
	(No symbol) [0x0067980A]
	(No symbol) [0x00678B10]
	(No symbol) [0x00677F65]
	(No symbol) [0x00677EFC]
	(No symbol) [0x00676BF6]
	(No symbol) [0x00677358]
	(No symbol) [0x00686542]
	(No symbol) [0x006DDB91]
	(No symbol) [0x006CAECC]
	(No symbol) [0x006DD57C]
	(No symbol) [0x006CACC6]
	(No symbol) [0x006A6F68]
	(No symbol) [0x006A80CD]
	GetHandleVerifier [0x00A53832+2506274]
	GetHandleVerifier [0x00A89794+2727300]
	GetHandleVerifier [0x00A8E36C+2746716]
	GetHandleVerifier [0x00886690+617600]
	(No symbol) [0x0077C712]
	(No symbol) [0x00781FF8]
	(No symbol) [0x007820DB]
	(No symbol) [0x0078C63B]
	BaseThreadInitThunk [0x775600F9+25]
	RtlGetAppContainerNamedObjectPath [0x77E17BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77E17

In [12]:
scraped10 = scrape_comments(df_yt10)
scraped10.to_csv('df_yt_scraped10.csv', index = True)


https://www.youtube.com/watch?v=1wCy9ofcynU done
Cookie pop-up not loaded
https://www.youtube.com/shorts/SZOhzgvzYk4 done
https://www.youtube.com/watch?v=CjurW8ztOb4 done
https://www.youtube.com/watch?v=WkiKfK3RnNk done
Cookie pop-up not loaded
https://www.youtube.com/shorts/7cAjVdnUmi8 done
https://www.youtube.com/watch?v=wiscimXGqxg done
Cookie pop-up not loaded
https://www.youtube.com/shorts/OtTioc-G3K8 done
Cookie pop-up not loaded
https://www.youtube.com/shorts/HxyiAF1D36s done
Cookie pop-up not loaded
https://www.youtube.com/shorts/EyUy64yLKZ8 done
Cookie pop-up not loaded
https://www.youtube.com/watch?v=XslkOncDunc done
https://www.youtube.com/watch?v=7A7ZTZyoHhU done
Cookie pop-up not loaded
https://www.youtube.com/shorts/GpAS21lHkSY done
https://www.youtube.com/watch?v=aERMkp0SE5o done
https://www.youtube.com/watch?v=I-EIVlHvHRM done
https://www.youtube.com/watch?v=_Rs80XA76FM done
https://www.youtube.com/watch?v=0pwzEwvQO3k done
Cookie pop-up not loaded
https://www.youtube.co

In [14]:
df_yt12  = df_yt.iloc[234:236].reset_index()

In [15]:
df_yt12

,index,Unnamed: 0,Name,views,duration,date,description,link
0,234,234,YOU are training Artificial Intelligence RIGHT...,161,Unknown,3 months ago,Your data is used to train Artificial Intellig...,https://www.youtube.com/shorts/9N2cGgP5SYk
1,235,235,Artificial Intelligence Told Me To Invent This...,"452,017","9 minutes, 23 seconds",2 months ago,Unnecessary Inventions is the brainchild of pr...,https://www.youtube.com/watch?v=aDoEMGUHgI0


In [23]:
def scrape_comments(df):
    comments = []

    for i, row in df.iterrows():
        url = row['link']
        options = webdriver.ChromeOptions()
        options.add_argument("--incognito")
        options.add_argument("--disable-gpu")

        driver = webdriver.Chrome(options=options)
        driver.get(url)
        time.sleep(30)
        try:
            wait = WebDriverWait(driver, 10)
            cookie_consent = wait.until(EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Accept the use of cookies and other data for the purposes described"]')))
            cookie_consent.click()
        except TimeoutException:
            print('Cookie pop-up not loaded')

        time.sleep(30)
        
        driver.execute_script("window.scrollBy(0, 500);")

        try:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.XPATH, '//ytd-comment-thread-renderer')))
            time.sleep(30)
            while True:
                scroll_height = 10
                document_height_before = driver.execute_script("return document.documentElement.scrollHeight")
                driver.execute_script(f"window.scrollTo(0, {document_height_before + scroll_height});")
                time.sleep(2)
                document_height_after = driver.execute_script("return document.documentElement.scrollHeight") 
                if document_height_after == document_height_before:
                    break

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            for comment in soup.find_all('ytd-comment-thread-renderer'):
                text = comment.find('yt-formatted-string', {'class': 'style-scope ytd-comment-renderer'}).text
                likes = comment.find('span', {'id': 'vote-count-left'}).text.strip() if comment.find('span', {'id': 'vote-count-left'}) else 0
                replies = comment.find('button', {'class': 'yt-spec-button-shape-next yt-spec-button-shape-next--text yt-spec-button-shape-next--call-to-action yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--align-by-text'})
                if replies:
                    num_comments = replies.get('aria-label').split()[0]
                else:
                    num_comments = 0
                username_element = comment.find('a', {'id': 'author-text'}).find('span', {'class': 'style-scope ytd-comment-renderer'})
                if username_element:
                    username = username_element.text.replace('[','').replace(']','').replace('\n','')
                else:
                    username = ''
                link = url
                try:
                    date = comment.find('a', class_='yt-simple-endpoint style-scope yt-formatted-string')
                except:
                    date = 'Unknown'


                comments.append([username, text, likes, num_comments, date, link])
        except TimeoutException:
            comments.append(["ERROR", 'ERROR', 'ERROR', 'ERROR', 'ERROR', url])
        finally:
            driver.quit()
            print(f'{url} done')

        df_comments = pd.DataFrame(comments, columns=['Username', 'Comment', 'Likes', 'Replies', 'Date', 'link'])

    return df_comments

In [24]:
scrape_comments(df_yt12)

Cookie pop-up not loaded
https://www.youtube.com/shorts/9N2cGgP5SYk done
https://www.youtube.com/watch?v=aDoEMGUHgI0 done


,Username,Comment,Likes,Replies,Date,link
0,ERROR,ERROR,ERROR,ERROR,ERROR,https://www.youtube.com/shorts/9N2cGgP5SYk
1,,"BTW, here is that poem about why Drake keeps b...",412,47,[3 months ago (edited)],https://www.youtube.com/watch?v=aDoEMGUHgI0
2,Anonymous 73,I’ve always thought this channel was less of “...,793,18,[3 months ago],https://www.youtube.com/watch?v=aDoEMGUHgI0
3,Nolo,Y’know what’s even crazier? You could’ve asked...,202,10,[2 months ago],https://www.youtube.com/watch?v=aDoEMGUHgI0
4,Talking Nerdy With CJ,Hint: You actually need a way to PULL down the...,288,14,[3 months ago],https://www.youtube.com/watch?v=aDoEMGUHgI0
...,...,...,...,...,...,...
476,HUDBRO,first,1,1,[3 months ago],https://www.youtube.com/watch?v=aDoEMGUHgI0
477,Nokia41,W*,0,1,[3 months ago],https://www.youtube.com/watch?v=aDoEMGUHgI0
478,CowboyVR🦍,First,1,1,[3 months ago],https://www.youtube.com/watch?v=aDoEMGUHgI0
479,Pancake,Lukas Eicher is the first comment,0,1,[3 months ago],https://www.youtube.com/watch?v=aDoEMGUHgI0
